# 03. Monte Carlo like Approach

## Naive Deep Learning

TicTacToe와 다르게 오목은 상태가 매우 많고
행동도 매우 많습니다. 판의 크기를 $9 \times 9$
로만 한정한다고 해도 $3^{81}$ 정도의 경우가 나오지만,
이 중 10%를 훑는 것조차 현실적으로 불가능할 정도입니다.
이전에 TicTacToe에서 구현한 DNN은 틱택토의 판 상태를
통째로 받아서 이를 바탕으로 행동을 결정하는데, 이 경우는
적어도 5분 정도 학습을 시켜도 loss가 빠르게 줄어드는
시점이 보이며, 알고리즘을 섞어서 exploration을
할 경우 알고리즘과 비교적 비슷한 행동을 하도록 만드는 것이
가능합니다.

하지만 오목의 경우에는 알고리즘만으로 exploration을
시도하더라도 loss가 줄어드는 상황을 보기 힘듭니다.
아래가 그 구현이며 실제로 매우 안 됩니다.
(https://github.com/lumiknit/mock5-q)

이 때문에 단순히 오목의 판을 넣고 Q-function의 값을
갱신하는 것 외에 더 효율적인 방법을 찾아야 합니다.

## Decision Tree

Decision tree는 각 정점이 상태를 나타내고
간선은 해당 상태에서 의사 결정을 통해 분기되는
경로를 나타내는 트리입니다.
이 상태라는 것이 무엇을 나타내는가는 맥락에 따라 다르지만,
게임의 경우에는 보통 현재 판의 상태를 포함하게 됩니다.

예를 들자면 체스의 경우에는 살아있는 기물들의 배치가 될
것이고, 바둑이나 오목은 돌의 배치가 됩니다.

일부 게임의 경우에는 상태만을 저장하게 되면 트리가 아니게
될 수 있습니다. 예를 들어서 체스에서 반복된 움직임에
의한 스테일메이트 규칙을 제외하면 같은 상태를 무한히
반복하는 것이 가능하며, 바둑에서 패를 따내는 것에
제한을 없애면 마찬가지로 같은 상태가 무한히 반복될
수 있습니다. 이러한 의사결정은 곳 결정 그래프에서
사이클을 만들어내는 문제가 있습니다. 다만, 위
같은 규칙들이 있거나, 오목처럼 strictly increasing
하는 경우에는 크게 고려할 필요가 없습니다.

만약에 효율적인 시간 내에 decision tree을 만들어내고
그 tree에서 필요한 범위를 탐색할 수 있다면 필승법을
찾아낼 수 있습니다.

오목의 경우를 예로 들자면 일단 판을 가득채우면 게임이
무조건 끝나므로 decision tree는 finite합니다.
이 상황에서 inductive하게 판단할 수 있는데,
만약 자기보다 낮은 depth의 유불리가 모두
판별이 되었다면, 자신의 자식은 모두 상대의 차례이므로,
가능한 결정 중 상대에게 가장 불리한 선택지를 고르는 것이
자신에게는 유리한 선택입니다.
더 나아가, 만약 자기보다 낮은 depth의 모든 노드가
'현 상태에서 자신이 둘 차례면 승리하는지 패배하는지'
여부가 모두 기록이 되었다면,
자신은 자식들 중 패배하는 기록이 하나라도 있다면
그 결정을 하여서 상대를 패배시킬 수 있으므로
자신은 승리할 수밖에 없으며,
반대로 그러한 수가 없으면 자신은 패배하게 됩니다.

하지만 이 decision tree의 크기는 가능한 판의 상태의
수와 같으므로, 일부를 컴퓨터에 저장하는 것도 힘들 정도로
거대합니다.

## Randomized Algorithm - Monte Carlo Algorithm

효율적인 시간 내에 정확한 답을 얻어내는 것은
생각보다 많은 문제에서 쉽지 않습니다.
이 때문에 결정적이지 않은 알고리즘을 사용해서
답을 근사하려고 하는 경우가 종종 있습니다.
이런 알고리즘을 probabilistic algorithm
(확률적 알고리즘) 또는 
randomized algorithm (무작위 알고리즘)
이라고 합니다.

Randomized algorithm에서 가장 잘 알려져 있는
방법은 Monte Carlo method와
Las Vegas algorithm입니다.
두 알고리즘 모두 무작위로 임의의 값을 뽑은 뒤
그 값을 이용하여 원하는 값을 찾아냅니다.
이 둘의 차이는 어떤 부분에서
무작위인지 아닌지를 나누게 되는데,
Monte Carlo는 실행 시간을 특정 시간으로 bound하지만
Las Vegas는 정확도에 bound를 하게 됩니다.

가장 대표적인 Monte Carlo method의 적용 사례로,
원주율을 근사하는 방법이 있겠습니다.
원의 넓이는 변의 길이와 원주율의 곱으로 이루어지고
임의의 점이 원 내부에 들어가는지 여부는 손쉽게
알 수 있기 때문에 가능한 방법입니다.

In [1]:
import numpy as np

def calc_pi_by_monte_carlo(num_tries):
  num_pt_in_circle = 0
  for i in range(num_tries):
    x = np.random.uniform()
    y = np.random.uniform()
    if x * x + y * y <= 1.0:
      num_pt_in_circle += 1
  area = num_pt_in_circle / num_tries # = 0.25 pi
  return 4 * area

In [2]:
for i in [10, 100, 10000, 1000000]:
  print("w/ {} points = {}".format(i, calc_pi_by_monte_carlo(i)))

w/ 10 points = 2.8
w/ 100 points = 3.36
w/ 10000 points = 3.142
w/ 1000000 points = 3.13862


위 결과를 보면 샘플이 많아지면 많아질수록
원주율에 근사하는 것을 볼 수 있습니다.
이렇듯 Monte Carlo method는 원하는 값이 정확히
어떻게 계산되는지 몰라도, 근사를 하는 전략이 되어줍니다.

주의할 점은 임의로 샘플링을 하여서 원하는 값으로 수렴한다는
것이 증명이 되어야하고, 그 방식에 맞는 확률분포로
무작위 값을 뽑아야 한다는 것입니다.
위 원주율 계산의 경우 만약 uniform distribution
이 아닌 normal distribution을 부여하게 되면
일종의 특정 영역에 가중치를 준 것이 되어
제대로 계산되지 않습니다.

In [3]:
import numpy as np

def calc_pi_by_monte_carlo_with_normal(num_tries):
  num_pt_in_circle = 0
  for i in range(num_tries):
    x = np.random.normal()
    x = max(x, -x)
    x = min(x, 1)
    y = np.random.normal()
    y = max(y, -y)
    y = min(y, 1)
    if x * x + y * y <= 1.0:
      num_pt_in_circle += 1
  area = num_pt_in_circle / num_tries # = we wish 0.25 pi
  return 4 * area

TRIES = 100000
pi_with_uniform = calc_pi_by_monte_carlo(TRIES)
pi_with_normal = calc_pi_by_monte_carlo_with_normal(TRIES)

print("PI from uniform distr = {}".format(pi_with_uniform))
print("PI from normal distr  = {}".format(pi_with_normal))

PI from uniform distr = 3.14404
PI from normal distr  = 1.57108


## Monte Carlo Tree Search

Monte Carlo Tree Search는
decision tree를 정확하게 구하는 대신에
Monte Carlo method를 이용하여 무작위 값을 바탕으로
근사해나가자는 것입니다.
여기서 무작위 값은 임의의 결정이 되며,
근사해나가는 것은 해당 상태에서의 승률
(정확히는 승패에 관한 통계로, 전체 게임 횟수와
그 중 승리한 횟수)입니다.

예를 들어서 오목의 경우는 아래와 같이 진행됩니다.

0. 각 상태에는 해당 상태를 지나면서 만약 모두 이겼으면 얻을 수 있는 '최대 점수'와 실제 게임 중 얻은 '실제 점수'를 저장.
1. 임의의 기보를 생성.
2. 기보의 마지막에 승패에 따른 점수를 부여. 예를 들어서 승리할 경우에 1, 무승부거나 패해한 경우에 0
3. 해당 점수를 바탕으로 역전파. 우선 '최대 점수'에는 가능한 최대 점수인 1을 더함. Leaf node는 2에서 계산한 점수가 '실제 점수'에 더해지며, parent node에는 child node의 승패여부를 뒤집어서 점수를 계산해서 '실제 점수'에 더하는 것을 반복.

즉, 임의의 기보에 대해 이긴 플레이어의 상태에 점수를 더하는 것을 반복하게 됩니다.
만약에 이 트리가 충분히 수렴했다면, 자신의 승률이 최대화
되는 (= 상대의 승률이 최소화 되는) 선택지가 경험적으로
유리한 선택지가 되는 것입니다.

다만, 저희는 각 선택에 가중치 없이 골고루 훑어서 가장
높은 확률의 선택지를 고르고 싶은 것이다보니, 모든 기보를
빠짐없이, 그러면서도 중복되지는 않도록 훑으며 계산을 하는
것이 가장 정확합니다.
문제는 실제로는 빠짐없이 모든 기보를 훑는 것이
너무 오래 걸리고, 각 기보를
uniform하게 추출하는 것도 힘든 상황이 많습니다.
그래서 적당한 시간에 끊을 수 있으면서도, 주된 상황들을
커버할 수 있는 적당한 방법이 필요합니다.

이 때문에 실제 Monte Carlo Tree Search는 다음과
같이 이루어집니다.

1. 현재 존재하는 Tree에 대해 가장 유리한 수를 선택(보통은 특정 정책을 따름)하면서 Leaf까지 내려갑니다.
2. 만약 Leaf까지 내려왔는데 게임이 끝나지 않으면 기본 정책을 따라 트리를 탐색합니다.
3. 최종적으로 게임이 종료되었다면 역전파를 합니다.
4. 이런 방식으로 트리를 충분히 갱신한 뒤에 최적의 수를 선택합니다.

여기서 트리를 내려가는 경우에는 단순히 혼자서 게임을
진행하면서 학습할 수도 있지만,
다른 상대를 두고 번갈아가면서 학습을 할 수도 있습니다.

## 다시 TicTacToe

TicTacToe는 기보의 수는 많아도 $9! = 362880$개 이하입니다. (9수까지 못 가고 끝날 수 있으므로) 
특히 첫 수는 회전을 시켜서 상단, 좌상단, 중앙 셋 중
하나로 만들 수 있으므로 $3 \cdot 8! = 120960$
으로 경우가 매우 적습니다.
그리고 가능한 상태의 수도 많아봐야
$222211111_{(3)} + 1 = 19684$개이므로
마음만 먹으면 decision tree를 전부 생성하는 것도
가능합니다.

다만, 여기서는 tree 전체를 생성하는 대신에, 게임을 진행할 수 있는 일부 횟수를 제한하여
Monte Carlo Tree Search를 한 뒤에
결과를 비교해봅니다.

### 틱택토 및 알고리즘

이전에 사용한 TicTacToe를 사용합니다.


In [4]:
import numpy as np

class Game:
  def __init__(self):
    self.board = [0] * 9
    self.turn = 1
    self.turns = 0
    self.foul = False

  def clone(self):
    g = Game()
    for i in range(9):
      g.board[i] = self.board[i]
    g.turn = self.turn
    g.turns = self.turns
    return g

  def rotate(self):
    # Clone and rotate clockwise
    g = Game()
    for r in range(3):
      for c in range(3):
        g.board[3 * r + c] = self.board[3 * (2 - c) + r]
    g.turn = self.turn
    g.turns = self.turns
    return g

  def place(self, idx):
    # Place stone at board[idx]
    # Iff the player cannot place a stone at idx, return False
    if self.board[idx] != 0:
      self.foul = True
      return False
    self.board[idx] = self.turn
    self.turn = 3 - self.turn
    self.turns += 1
    return True

  def check(self, i0, i1, i2):
    # Check whether 3 stones of a same color are placed in i0, i1, i2
    v = self.board[i0]
    if 0 != v and v == self.board[i1] and v == self.board[i2]:
      return v
    else: return None

  def check_win(self):
    # If some player 1 or 2 win, return 1 or 2
    # If they draw, return 0
    # Otherwise return None
    if self.foul:
      return 3 - self.turn
    r = None
    r = r or self.check(0, 1, 2) or self.check(3, 4, 5) or self.check(6, 7, 8)
    r = r or self.check(0, 3, 6) or self.check(1, 4, 7) or self.check(2, 5, 8)
    r = r or self.check(0, 4, 8) or self.check(2, 4, 6)
    if self.turns >= 9: r = r or 0
    return r
  
  def to_numpy(self, player):
    # Create 3 * 3 * 3 np array
    a = []
    for i in range(3):
      p = i
      if player == 2: p = (3 - p) % 3
      b = []
      for r in range(3):
        col = []
        for c in range(3):
          col.append(1. if self.board[r * 3 + c] == p else 0.)
        b.append(col)
      a.append(b)
    return np.array(a)

  def __str__(self):
    ch_arr = ['.', 'O', 'X']
    s = "---\nTurn: {} ({})\n".format(self.turn, ch_arr[self.turn])
    s += " | 0 1 2\n-+------"
    for r in range(3):
      s += "\n{}| ".format(r * 3)
      for c in range(3):
        s += "{} ".format(ch_arr[self.board[r * 3 + c]])
    return s

  def play(self, f1, f2, no_print=False):
    if no_print: p = lambda x: x
    else: p = print
    # Play game using two input functions
    if f1 == None: f1 = user_input
    if f2 == None: f2 = user_input
    tfn = [None, f1, f2]
    while self.turns < 9: # While empty cell exists
      p(str(self))
      # Take an index of cell to put a stone
      x = tfn[self.turn](self)
      p("{}P INPUT = {}".format(self.turn, x))
      # Check foul move.
      if type(x) != int or x < 0 or x >= 9 or self.board[x] != 0:
        p(str(self))
        p("Player {} put a stone on a non-empty cell!, Player {} win!" \
            .format(self.turn, 3 - self.turn))
        return
      # Place a stone
      self.place(x)
      # Check game is done
      w = self.check_win()
      if w == 0: # Draw
        p(str(self))
        p("Draw")
        return
      elif w != None: # Player `w` win
        p(str(self))
        p("Player {} win!".format(w))
        return
    p(str(self))
    p("Draw")

def user_input(game):
  i = None
  while i == None:
    try:
      # Read a index of cell (0~8)
      x = input("Idx(0~8) > ")
      i = int(x)
    except:
      print("Wrong Input!")
  return i

In [5]:
import random

def ttt_alg(game):
  # Simple strategy to win TicTacToe
  def c(i):
    # Check there are 2 same-color stones in a row
    v = max([game.board[x] for x in i])
    if v == 0: return None
    cnt = 0
    e = None
    for j in range(3):
      if v == game.board[i[j]]: cnt += 1
      else: e = i[j]
    if cnt != 2 or game.board[e] != 0: return None
    return e
  # The 1st or 2nd stone must be at the center or a corner
  if game.turns <= 1:
    if game.board[4] == 0: return 4
    else: return [0, 2, 6, 8][random.randrange(4)]
  # Otherwise, find there are 2 stones in a row
  r = c([0, 1, 2]) or c([3, 4, 5]) or c([6, 7, 8])
  r = r or c([0, 3, 6]) or c([1, 4, 7]) or c([2, 5, 8])
  r = r or c([0, 4, 8]) or c([2, 4, 6])
  if r != None: return r
  # Otherwise, just pick a random position
  for x in [4, 0, 2, 8, 6, 1, 7, 3, 5]:
    if game.board[x] == 0: return x

여기서
각 틱택토 상태를 정수로 인코딩하는 함수를 만들어줍니다.


In [6]:
def bd2int(bd):
  v = 0
  for i in range(9):
    v = v * 3 + bd[i]
  return v

def game2int(game):
  return bd2int(game.board)

def int2bd(st):
  a = [0] * 9
  for i in range(9):
    a[8 - i] = st % 3
    st //= 3
  return a

In [7]:
g = Game()
print(game2int(g), g)
g.place(4)
print(game2int(g), g)
g.place(2)
print(game2int(g), g)
g.place(0)
print(game2int(g), g)
g.place(8)
g.place(6)
g.place(3)
print(game2int(g), g)

0 ---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
81 ---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . . 
1539 ---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . X 
3| . O . 
6| . . . 
8100 ---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| . O . 
6| . . . 
8597 ---
Turn: 1 (O)
 | 0 1 2
-+------
0| O . X 
3| X O . 
6| O . X 


In [8]:
print(int2bd(8597))

[1, 0, 2, 2, 1, 0, 1, 0, 2]


아래에서 트리를 생성합니다.
트리의 각 노드에는 방문한 횟수와 승리한 횟수를 저장합니다.
탐색하는 정책은 단순히 승률이 가장 좋았던 경로를 택합니다.

In [9]:
class Tree:
  def __init__(self):
    self.tbl = {}
    self.turns = {}
    self.children = {}

  def find_children(self, st):
    if st in self.children: return self.children[st], self.turns[st]
    a = int2bd(st)
    c = 1
    for i in range(9):
      if a[i] != 0: c = 3 - c
    d = [None] * 9
    for i in range(9):
      if a[i] == 0:
        a[i] = c
        d[i] = bd2int(a)
        a[i] = 0
    self.turns[st] = c
    self.children[st] = d
    return d, c

  def get_win_rate(self, st):
    if st in self.tbl:
      w, n = self.tbl[st]
      return w / n
    else:
      return 0.5

  def update(self, st, inc):
    if st in self.tbl:
      w, n = self.tbl[st]
      self.tbl[st] = (w + inc, n + 1)
    else:
      self.tbl[st] = (inc, 1)

  def backpropagate(self, history, v = None):
    if v is None:
      v = len(history) % 2
    for h in history:
      self.update(h, v)
      v = 1 - v

  def make_decision(self, st):
    d, c = self.find_children(st)
    maxv = -float('inf')
    maxi = -1
    for i in range(9):
      if d[i] is not None:
        v = self.get_win_rate(d[i])
        if v > maxv or (v == maxv and np.random.uniform() < 0.5):
          maxv = v
          maxi = i
    return maxi

In [10]:
def make_mcts_agent(tree):
  def agent(game):
    st = game2int(game)
    print(list(map(lambda x: tree.get_win_rate(x), tree.find_children(st)[0])))
    return tree.make_decision(game2int(game))
  return agent

### 자습

기본적으로는 자기 자신가 두되,
일정 확률로 랜덤한 탐색을 시도합니다.

In [11]:
import random
def independent_learn(num_games):
  t = Tree()
  for n in range(num_games):
    g = Game()
    st = game2int(g)
    h = [game2int(g)]
    winner = None
    while winner is None:
      if random.random() < 0.3:
        i = random.randint(0, 8)
        for off in range(9):
          j = (off + i) % 9
          if g.board[i] == 0:
            i = j
            break
      else:
        i = t.make_decision(st)
      g.place(i)
      st = game2int(g)
      h.append(st)
      winner = g.check_win()
    t.backpropagate(h, 0.5 if winner == 0 else None)
    if n % 50000 == 50000 - 1:
      print("{:7.3f}% Done".format(100.0 * ((n + 1) / num_games)))
  return t

In [12]:
t = independent_learn(3000)
print("Covered # of states: {}".format(len(t.tbl)))

Covered # of states: 2393


In [13]:
Game().play(ttt_alg, make_mcts_agent(t))

---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
1P INPUT = 4
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . . 
[0.4062273714699493, 0.24305555555555555, 0.26344086021505375, 0.3302752293577982, 0.5, 0.31333333333333335, 0.38461538461538464, 0.28125, 0.3531468531468531]
2P INPUT = 0
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X . . 
3| . O . 
6| . . . 
1P INPUT = 2
---
Turn: 2 (X)
 | 0 1 2
-+------
0| X . O 
3| . O . 
6| . . . 
[0.5, 0.0, 0.5, 0.25, 0.5, 0.5, 0.6222222222222222, 0.25, 0.2]
2P INPUT = 6
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X . O 
3| . O . 
6| X . . 
1P INPUT = 3
---
Turn: 2 (X)
 | 0 1 2
-+------
0| X . O 
3| O O . 
6| X . . 
[0.5, 0.3333333333333333, 0.5, 0.5, 0.5, 0.4666666666666667, 0.5, 0.25, 0.0]
2P INPUT = 5
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X . O 
3| O O X 
6| X . . 
1P INPUT = 8
---
Turn: 2 (X)
 | 0 1 2
-+------
0| X . O 
3| O O X 
6| X . O 
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
2P INPUT = 7
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X 

In [14]:
Game().play(make_mcts_agent(t), ttt_alg)

---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
[0.5868055555555556, 0.5431034482758621, 0.5614035087719298, 0.5669291338582677, 0.6566265060240963, 0.5672268907563025, 0.5618556701030928, 0.610236220472441, 0.5702702702702702]
1P INPUT = 4
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . . 
2P INPUT = 2
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . X 
3| . O . 
6| . . . 
[0.8333333333333334, 0.375, 0.5, 0.3333333333333333, 0.5, 0.823076923076923, 0.5, 0.6923076923076923, 0.6428571428571429]
1P INPUT = 0
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| . O . 
6| . . . 
2P INPUT = 8
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O . X 
3| . O . 
6| . . X 
[0.5, 0.2, 0.5, 0.6666666666666666, 0.5, 0.5, 0.4, 0.0, 0.5]
1P INPUT = 3
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| O O . 
6| . . X 
2P INPUT = 5
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O . X 
3| O O X 
6| . . X 
Player 2 win!


약 3000판의 플레이를 하게 되면 2500개 정도의 기보를 커버하게 되며, 대략 이 쯤부터 알고리즘과 무승부를 낼 수 있게 됩니다.

In [15]:
t = independent_learn(1000000)
print("Covered # of states: {}".format(len(t.tbl)))

  5.000% Done
 10.000% Done
 15.000% Done
 20.000% Done
 25.000% Done
 30.000% Done
 35.000% Done
 40.000% Done
 45.000% Done
 50.000% Done
 55.000% Done
 60.000% Done
 65.000% Done
 70.000% Done
 75.000% Done
 80.000% Done
 85.000% Done
 90.000% Done
 95.000% Done
100.000% Done
Covered # of states: 5377


In [16]:
Game().play(ttt_alg, make_mcts_agent(t))

---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
1P INPUT = 4
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . . 
[0.4331034741221002, 0.301353226925746, 0.43375141888223806, 0.295502716867702, 0.5, 0.29867516987434994, 0.42793088786672334, 0.30084992432180696, 0.4321864224537466]
2P INPUT = 2
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . X 
3| . O . 
6| . . . 
1P INPUT = 0
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| . O . 
6| . . . 
[0.5, 0.15832205683355885, 0.5, 0.17529107373868047, 0.5, 0.23716012084592145, 0.13215859030837004, 0.16330935251798562, 0.5305025882969662]
2P INPUT = 8
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O . X 
3| . O . 
6| . . X 
1P INPUT = 5
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| . O O 
6| . . X 
[0.5, 0.18693693693693694, 0.5, 0.5019640779788516, 0.5, 0.5, 0.20967741935483872, 0.21049046321525886, 0.5]
2P INPUT = 3
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O . X 
3| X O O 
6| . . X 
1P INPUT = 6
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 

In [17]:
Game().play(make_mcts_agent(t), ttt_alg)

---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
[0.5869054295863045, 0.5588778742374472, 0.5840691420518278, 0.5594824865888293, 0.6162251123209542, 0.5616656162264367, 0.5837541256452933, 0.5630093047448419, 0.5849279538904899]
1P INPUT = 4
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . . 
2P INPUT = 8
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . X 
[0.549, 0.5850642927794263, 0.5909090909090909, 0.572208228379513, 0.5, 0.6236459921367247, 0.6060498220640569, 0.6252547801611181, 0.5]
1P INPUT = 7
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . O X 
2P INPUT = 1
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . X . 
3| . O . 
6| . O X 
[0.5359195402298851, 0.5, 0.6077616077616078, 0.6009253903990746, 0.5, 0.5786219081272085, 0.3376383763837638, 0.5, 0.5]
1P INPUT = 2
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . X O 
3| . O . 
6| . O X 
2P INPUT = 6
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . X O 
3| . O . 
6| X O X 
[0.5810356422326832, 0.5, 0.5, 0.59

게임 횟수를 약 5배로 늘려보아도 커버하는 경우가 확연히 늘어나지는 않습니다.

### 힌트 학습

In [18]:
import random
def hint_learn(num_games):
  t = Tree()
  epsilon = 0.8
  ep_dec = 0.99
  for n in range(num_games):
    g = Game()
    st = game2int(g)
    h = [game2int(g)]
    winner = None
    epsilon *= ep_dec
    while winner is None:
      if random.random() < epsilon:
        i = ttt_alg(g)
      else:
        i = t.make_decision(st)
      g.place(i)
      st = game2int(g)
      h.append(st)
      winner = g.check_win()
    t.backpropagate(h, 0.5 if winner == 0 else None)
    if n % 50000 == 50000 - 1:
      print("{:7.3f}% Done".format(100.0 * ((n + 1) / num_games)))
  return t

In [19]:
t = hint_learn(200)
print("Covered # of states: {}".format(len(t.tbl)))
print(t)

Covered # of states: 269


In [20]:
Game().play(ttt_alg, make_mcts_agent(t))

---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
1P INPUT = 4
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . . 
[0.4609375, 0.0, 0.3611111111111111, 0.0, 0.5, 0.3, 0.3333333333333333, 0.25, 0.3125]
2P INPUT = 0
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X . . 
3| . O . 
6| . . . 
1P INPUT = 2
---
Turn: 2 (X)
 | 0 1 2
-+------
0| X . O 
3| . O . 
6| . . . 
[0.5, 0.0, 0.5, 0.3333333333333333, 0.5, 0.5, 0.5, 0.0, 0.0]
2P INPUT = 6
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X . O 
3| . O . 
6| X . . 
1P INPUT = 3
---
Turn: 2 (X)
 | 0 1 2
-+------
0| X . O 
3| O O . 
6| X . . 
[0.5, 0.0, 0.5, 0.5, 0.5, 0.4666666666666667, 0.5, 0.25, 0.0]
2P INPUT = 5
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X . O 
3| O O X 
6| X . . 
1P INPUT = 8
---
Turn: 2 (X)
 | 0 1 2
-+------
0| X . O 
3| O O X 
6| X . O 
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
2P INPUT = 1
---
Turn: 1 (O)
 | 0 1 2
-+------
0| X X O 
3| O O X 
6| X . O 
1P INPUT = 7
---
Turn: 2 (X)
 | 0 1 2
-+------
0| X X O 
3| O O X

In [21]:
Game().play(make_mcts_agent(t), ttt_alg)

---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
[0.5, 0.5, 0.5, 0.5, 0.59, 0.5, 0.5, 0.5, 0.5]
1P INPUT = 4
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . . 
2P INPUT = 6
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| X . . 
[0.6666666666666666, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
1P INPUT = 0
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . . 
3| . O . 
6| X . . 
2P INPUT = 8
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O . . 
3| . O . 
6| X . X 
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.59375, 0.5]
1P INPUT = 7
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . . 
3| . O . 
6| X O X 
2P INPUT = 1
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O X . 
3| . O . 
6| X O X 
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
1P INPUT = 3
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O X . 
3| O O . 
6| X O X 
2P INPUT = 5
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O X . 
3| O O X 
6| X O X 
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
1P INPUT = 2
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O X O 
3| O O X 
6| 

algorithm 자체를 힌트로 주고 기보를
누적할 경우에는 훨씬 적은 횟수로도
'알고리즘이 두는 방식'으로 학습을 시키는 것이 가능합니다.
(다만, 이 경우는 커버리지 자체가 넓지 않기 때문에
알고리즘과 다른 스타일로 두게 되면 매우 약해질 수 있습니다.)

## Beat the algorithm down

이번 파트를 마무리하기 전,
지금까지 사용한 틱택토 알고리즘을
무승부가 아니라 이기는 것은 불가능한지
확인하고 넘어가고자 합니다.

이를 위해서 모든 경우를 순회해도 되겠지만,
MCTS로 얼마나 반복하면 이 경우까지 커버가
되는지 확인해봅시다.

In [22]:
class Tree:
  def __init__(self):
    self.tbl = {}
    self.turns = {}
    self.children = {}

  def find_children(self, st):
    if st in self.children: return self.children[st], self.turns[st]
    a = int2bd(st)
    c = 1
    for i in range(9):
      if a[i] != 0: c = 3 - c
    d = [None] * 9
    for i in range(9):
      if a[i] == 0:
        a[i] = c
        d[i] = bd2int(a)
        a[i] = 0
    self.turns[st] = c
    self.children[st] = d
    return d, c

  def get_win_rate(self, st):
    if st in self.tbl:
      w, n = self.tbl[st]
      return w / n
    else:
      return 0.5

  def update(self, st, inc):
    if st in self.tbl:
      w, n = self.tbl[st]
      self.tbl[st] = (w + inc, n + 1)
    else:
      self.tbl[st] = (inc, 1)

  def backpropagate(self, history, w = 1.0):
    v = len(history) % 2
    for h in history:
      self.update(h, v * w)
      v = 1 - v

  def make_decision(self, st):
    d, c = self.find_children(st)
    maxv = -float('inf')
    maxi = -1
    for i in range(9):
      if d[i] is not None:
        v = self.get_win_rate(d[i])
        if v > maxv or (v == maxv and np.random.uniform() < 0.5):
          maxv = v
          maxi = i
    return maxi

In [23]:
import random
def ext_learn(num_games):
  t = Tree()
  epsilon = 1.0
  ep_dec = 0.9999
  min_ep = 0.3
  for n in range(num_games):
    epsilon *= ep_dec
    if epsilon < min_ep: epsilon = min_ep
    g = Game()
    st = game2int(g)
    h = [game2int(g)]
    winner = None
    while winner is None:
      if random.random() < epsilon:
        a = []
        for j in range(9):
          if g.board[j] == 0:
            a.append(j)
        i = a[random.randint(0, len(a) - 1)]
      elif random.random() < 0.1:
        i = ttt_alg(g)
      else:
        i = t.make_decision(st)
      g.place(i)
      st = game2int(g)
      h.append(st)
      winner = g.check_win()
    t.backpropagate(h, 0.0 if winner == 0 else 1.0)
    if n % 1000 == 1000 - 1:
      print("{:7.3f}% Done (#St = {})".format(100.0 * ((n + 1) / num_games), len(t.tbl)))
  return t

In [24]:
t = ext_learn(10000)
print("Covered # of states: {}".format(len(t.tbl)))

 10.000% Done (#St = 3166)
 20.000% Done (#St = 4297)
 30.000% Done (#St = 4837)
 40.000% Done (#St = 5095)
 50.000% Done (#St = 5206)
 60.000% Done (#St = 5272)
 70.000% Done (#St = 5308)
 80.000% Done (#St = 5329)
 90.000% Done (#St = 5342)
100.000% Done (#St = 5349)
Covered # of states: 5349


In [25]:
Game().play(make_mcts_agent(t), ttt_alg)

---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
[0.5818673883626523, 0.6055312954876274, 0.6048850574712644, 0.5140845070422535, 0.7235715938006014, 0.5494186046511628, 0.5964432284541724, 0.550326797385621, 0.6006051437216339]
1P INPUT = 4
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . . 
2P INPUT = 8
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . X 
[0.625, 0.639344262295082, 0.6585365853658537, 0.7209302325581395, 0.5, 0.7555555555555555, 0.7105263157894737, 0.8192090395480226, 0.5]
1P INPUT = 7
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . O X 
2P INPUT = 1
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . X . 
3| . O . 
6| . O X 
[0.5, 0.5, 0.5, 0.6363636363636364, 0.5, 0.7, 0.3333333333333333, 0.5, 0.5]
1P INPUT = 5
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . X . 
3| . O O 
6| . O X 
2P INPUT = 3
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . X . 
3| X O O 
6| . O X 
[0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5714285714285714, 0.5, 0.5]
1P INPUT = 6
---
Tur

In [26]:
Game().play(ttt_alg, make_mcts_agent(t))

---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . . 
3| . . . 
6| . . . 
1P INPUT = 4
---
Turn: 2 (X)
 | 0 1 2
-+------
0| . . . 
3| . O . 
6| . . . 
[0.22448979591836735, 0.18269230769230768, 0.22718052738336714, 0.16447368421052633, 0.5, 0.14792899408284024, 0.22493887530562348, 0.19572953736654805, 0.1837837837837838]
2P INPUT = 2
---
Turn: 1 (O)
 | 0 1 2
-+------
0| . . X 
3| . O . 
6| . . . 
1P INPUT = 0
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| . O . 
6| . . . 
[0.5, 0.058823529411764705, 0.5, 0.05263157894736842, 0.5, 0.3333333333333333, 0.09090909090909091, 0.0, 0.373134328358209]
2P INPUT = 8
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O . X 
3| . O . 
6| . . X 
1P INPUT = 5
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| . O O 
6| . . X 
[0.5, 0.0, 0.5, 0.0, 0.5, 0.5, 0.08, 0.13333333333333333, 0.5]
2P INPUT = 7
---
Turn: 1 (O)
 | 0 1 2
-+------
0| O . X 
3| . O O 
6| . X X 
1P INPUT = 3
---
Turn: 2 (X)
 | 0 1 2
-+------
0| O . X 
3| O O O 
6| . X X 
Player 1 win!


위에서는 일부 학습 정책을 바꿔서 탐색하는 속도를 수정하였습니다.
원래 TicTacToe는 양측이 수를 완벽하게 읽으면 항상
무승부가 나올 수밖에 없는데, 여기서는 이기기 위한
수들만을 남기기 위해서 무승부에 대해 양측에
어떠한 점수도 주지 않았습니다.
아직 탐색을 하지 않았다면
최소한 0.5라는 점수는 주기 때문에, 이 경우
이미 진다고 확인한 곳보다는 방문하지 않은 정점을
먼저 탐색을 하게 됩니다.
추가적으로 랜덤 탐색을 할 때에도 남은 칸에 대해서
uniform한 확률로 선택이 되도록 수정을 하였습니다.

위와 같이 학습을 하였을 때 약 5000번 동안
5000개 가량의 상태를 커버하였고, 이후부터는 조금씩
방문하지 않은 일부 상태만을 확인할 뿐
급격한 변화는 없습니다.

아래에서 MCTS가 후공인 경우에 알고리즘을 이기는 것을
확인할 수 있는데, 이 부분에서 알고리즘이 고려하지 않은
경우 또는 버그가 있다는 것을 확인할 수 있습니다.

참고로 틱택토의 모든 경우의 수는 다음과 같습니다.

In [27]:
def gen_full_tree(t, st):
  if st not in t.children:
    d, z = t.find_children(st)
    g = Game()
    g.board = int2bd(st)
    if g.check_win() is None:
      for i in d:
        if i is not None:
          gen_full_tree(t, i)
  return t
print(len(gen_full_tree(Tree(), 0).children))

5478
